# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=4439026712#4310029237#4088494919
date='December 6, 2020'
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
import sys
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
bkp=pd.read_json(db_file)
if not LAST and not bkp[bkp.Info.astype(str).str.contains(date)].empty:
    sys.exit('Check if already exists in bkp')

In [4]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [5]:
ll[0]

,Unnamed: 0,Avg,Max
0,Speed,17.4km/h,91.4km/h
1,Calories,9905,9905
2,Elapsed Time,12:03:30,12:03:30


In [6]:
sgmt

,Name,Time,Speed,Power,Info
0,Viva Envigado - Barranquilla 9.89km 63m -0.6%,18:57,31.3km/h,272W,"{'Calories': {'Avg': '9905', 'Max': '9905'}, '..."
1,Carrefour Vegas- Puente de la 30 4.89km 37m ...,9:26,31.1km/h,263W,"{'Calories': {'Avg': '9905', 'Max': '9905'}, '..."
2,Retorno Ayura -Pte 12 sur 0.84km 7m -0.8%,1:42,30.0km/h,221W,"{'Calories': {'Avg': '9905', 'Max': '9905'}, '..."
3,12 sur (Aguacatala) - La 4 sur 0.98km 13m -...,1:50,32.3km/h,259W,"{'Calories': {'Avg': '9905', 'Max': '9905'}, '..."
4,12 sur (Aguacatala) - La 10 2.09km 19m -0.7%,3:56,32.0km/h,271W,"{'Calories': {'Avg': '9905', 'Max': '9905'}, '..."
...,...,...,...,...,...
171,NaN,NaN,10:40,27.5km/h,"{'Calories': {'Avg': '9905', 'Max': '9905'}, '..."
172,NaN,NaN,18:15,20.0km/h,"{'Calories': {'Avg': '9905', 'Max': '9905'}, '..."
173,NaN,NaN,14:33,18.7km/h,"{'Calories': {'Avg': '9905', 'Max': '9905'}, '..."
174,NaN,NaN,19:26,19.4km/h,"{'Calories': {'Avg': '9905', 'Max': '9905'}, '..."


In [7]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [8]:
df['istr']=df['Info'].astype(str)

In [9]:
df=df.drop_duplicates(['Name', 'Power', 'Speed', 'Time', 'istr']).drop('istr',axis='columns').reset_index(drop=True)

In [10]:
df.to_json(db_file)

### Save notebook here

In [11]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [12]:
!bash commit.sh

[master 32bea64] Barbosa Concepción Alto de la Virgen 5:09 AM on Sunday, February 14, 2021
 2 files changed, 637 insertions(+), 13 deletions(-)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 10.93 KiB | 339.00 KiB/s, done.
Total 5 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:restrepo/strava.git
   455fbb8..32bea64  master -> master


In [ ]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? 
